In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
deliveries = pd.read_csv("deliveries.csv")
matches = pd.read_csv("matches.csv")

In [48]:
deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [49]:
matches.head()

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [50]:
df = deliveries.groupby(by=["match_id","batting_team","bowling_team"])["total_runs"].sum()
df = pd.DataFrame(df).reset_index().set_index("match_id")


scores = pd.DataFrame(columns=["match_id","Teams","Toss Winner","Toss Decision","Team 1","Team 2","Team 1 Score","Team 2 Score"])
team_scores = {}
ctr = 0
for i in df.index.unique():
    #print(df.loc[i,"batting_team"])
    try:
        teams = df.loc[i,"batting_team"].values
        toss_winner = matches[matches["id"]==i]["toss_winner"].values[0]
        choice = matches[matches["id"]==i]["toss_decision"].values[0]
        #print("Toss Winning Team : ",toss_winner)
        #print("Choice of",toss_winner,":",choice)

        position_of_toss_winner = np.where(teams == toss_winner)[0][0]
        toss_win = teams[position_of_toss_winner]

        if toss_winner == teams[0]:
            if choice == "bat":
                first_batting_team = teams[0]
            elif choice == "field":
                first_batting_team = teams[1]
        else:
            if choice == "bat":
                first_batting_team = teams[1]
            elif choice == "field":
                first_batting_team = teams[0]
        second_batting_team = teams[1-np.where(teams==first_batting_team)[0][0]]
        p = teams[0] + " vs " + teams[1]
        scores.loc[ctr,"match_id"] = i
        scores.loc[ctr,"Teams"] = p
        scores.loc[ctr,"Team 1"] = first_batting_team
        scores.loc[ctr,"Team 2"] = second_batting_team
        particular_match = df.loc[i,]
        scores.loc[ctr,"Team 1 Score"] = particular_match[particular_match["batting_team"]==first_batting_team]["total_runs"].values[0]
        scores.loc[ctr,"Team 2 Score"] = particular_match[particular_match["batting_team"]==second_batting_team]["total_runs"].values[0]
        scores.loc[ctr,"Toss Winner"] = toss_winner
        scores.loc[ctr,"Toss Decision"] = choice
        #print(p)
        ctr += 1
        #break
    #print()
    except:
        batting_team = df.loc[i,"batting_team"]
        bowling_team = df.loc[i,"bowling_team"]

        toss_winner = matches[matches["id"]==i]["toss_winner"].values[0]
        choice = matches[matches["id"]==i]["toss_decision"].values[0]
        #print("Toss Winning Team : ",toss_winner)
        #print("Choice of",toss_winner,":",choice)

        #position_of_toss_winner = np.where(teams == toss_winner)[0][0]
        #toss_win = teams[position_of_toss_winner]
        
        scores.loc[ctr,"match_id"] = i
        scores.loc[ctr,"Teams"] = batting_team + " vs " + bowling_team
        scores.loc[ctr,"Team 1"] = batting_team
        scores.loc[ctr,"Team 2"] = bowling_team
        particular_match = df.loc[i,]
        scores.loc[ctr,"Team 1 Score"] = particular_match["total_runs"]
        scores.loc[ctr,"Team 2 Score"] = np.nan
        scores.loc[ctr,"Toss Winner"] = toss_winner
        scores.loc[ctr,"Toss Decision"] = choice
        ctr += 1

In [52]:
deliveries.head()


,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [56]:
#deliveries.groupby(by=["match_id","batting_team","bowling_team"])["total_runs"].sum()
w = pd.DataFrame(columns=["Batting Team","Bowling Team"])
ctr = 0
for i in deliveries.index.values[:-1]:
    if deliveries.loc[i,"batsman"] != deliveries.loc[i+1,"batsman"]:
        w.loc[ctr,"Batting Team"] = deliveries.loc[i,"batting_team"]
        w.loc[ctr,"Bowling Team"] = deliveries.loc[i,"bowling_team"]
        ctr += 1

In [58]:
wick = w.groupby(by=["Batting Team","Bowling Team"]).sum()
wick

In [41]:
print(deliveries.index.values[:-1])

[     0      1      2 ... 179074 179075 179076]


In [35]:
scores.head()

,match_id,Teams,Toss Winner,Toss Decision,Team 1,Team 2,Team 1 Score,Team 2 Score
0,1,Royal Challengers Bangalore vs Sunrisers Hyder...,Royal Challengers Bangalore,field,Sunrisers Hyderabad,Royal Challengers Bangalore,207,172
1,2,Mumbai Indians vs Rising Pune Supergiant,Rising Pune Supergiant,field,Mumbai Indians,Rising Pune Supergiant,184,187
2,3,Gujarat Lions vs Kolkata Knight Riders,Kolkata Knight Riders,field,Gujarat Lions,Kolkata Knight Riders,183,184
3,4,Kings XI Punjab vs Rising Pune Supergiant,Kings XI Punjab,field,Rising Pune Supergiant,Kings XI Punjab,163,164
4,5,Delhi Daredevils vs Royal Challengers Bangalore,Royal Challengers Bangalore,bat,Royal Challengers Bangalore,Delhi Daredevils,157,142


In [121]:
scores.head()

,match_id,batting_team,bowling_team,total_runs
0,1,Royal Challengers Bangalore,Sunrisers Hyderabad,172
1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,207
2,2,Mumbai Indians,Rising Pune Supergiant,184
3,2,Rising Pune Supergiant,Mumbai Indians,187
4,3,Gujarat Lions,Kolkata Knight Riders,183


In [98]:
matches.head()

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [96]:
df.head()

,batting_team,bowling_team,total_runs
match_id,,,
1,Royal Challengers Bangalore,Sunrisers Hyderabad,172
1,Sunrisers Hyderabad,Royal Challengers Bangalore,207
2,Mumbai Indians,Rising Pune Supergiant,184
2,Rising Pune Supergiant,Mumbai Indians,187
3,Gujarat Lions,Kolkata Knight Riders,183
